# Use case Batica

## Load data

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [ ]:
df_excel = pd.read_excel(
    'data_catalogue.xlsx')

## Vue macro Catalog

In [ ]:
print("Le catalogue contient {0} observations, dont {1} valeurs uniques".format(
    df_excel.shape[0], 
    df_excel['Filename'].nunique())
     )

In [ ]:
(df_excel
 .groupby(['Storage','Theme'])['Theme']
 .count()
 .unstack(level = 1)
 .fillna(0)
 .assign(total = lambda x: x.sum(axis = 1))
 .sort_values(by = 'total', ascending = False)
 .T
 .sort_values(by = 'GCS', ascending = False)
 .style
 .background_gradient(cmap='Blues')
 #.bar(subset=['total'], color='#d65f5f')
 .format("{:0}")
)

In [ ]:
(df_excel
 .groupby(['Theme', 'Filename'])['Filename']
 .nunique()
 .unstack(level = 1)
 .fillna(0)
 .assign(total = lambda x: x.sum(axis = 1))
 .sort_values(by = 'total', ascending = False)
 .T
 .sort_values(by = 'Trade', ascending = False)
 #.loc[lambda x: x['total'] > 1]
 .style
 .background_gradient(cmap='Blues')
 #.bar(subset=['total'], color='#d65f5f')
 .format("{:0}")
)

# Recherche avancée: Macro

### Code app

In [ ]:
(df_excel
               .groupby(['Theme', 'Filename'])['Filename']
               .nunique()
               .unstack(level=0)
 .fillna(0)
 #.droplevel('Filename', axis = 0)
 #.xs(('Accountancy'), level=('Theme'), axis=1)
 .loc[:,('Accountancy')]
 .loc[lambda x: x>0]
)

In [ ]:
def dropdown_theme_eventhandler(change):
    
    with output_theme:
        output_theme.clear_output()
        if (change.new == 'ALL'):
            df_ = (df_excel
               .groupby(['Theme', 'Filename'])['Filename']
               .nunique()
               .unstack(level=[0])
               .fillna(0)
               .loc[lambda x: x>0]
               #.loc[:,(theme)]
               #.assign(total=lambda x: x.sum(axis=1))
               #.sort_values(by='total', ascending=False)
               #.loc[lambda x: ~x['total'].isin([0])]
               )
            display(df_)
        else:
            df_ = (df_excel
               .groupby(['Theme', 'Filename'])['Filename']
               .nunique()
               .unstack(level=[0])
               .fillna(0)
               .loc[:,(change.new)]
                .loc[lambda x: x>0]
                   .reset_index()
               #.xs((change.new), level=('Theme'), axis=1)
               #.assign(total=lambda x: x.sum(axis=1))
               #.sort_values(by='total', ascending=False)
               #.loc[lambda x: ~x['total'].isin([0])]
               )
            display(df_)

# App Macro

In [ ]:
dropdown_theme = widgets.Dropdown(
    options = np.append([df_excel['Theme'].sort_values().unique()],
                        ['ALL']))
output_theme = widgets.Output()

dropdown_theme.observe(dropdown_theme_eventhandler, names='value')
display(dropdown_theme)

In [ ]:
display(output_theme)

# Recherche avancée: Micro

### Prepare App

In [ ]:
def return_df(theme, module, fournisseur):
    df_ = df_excel.loc[
        lambda x:
        (x['Theme'].isin([theme]))
        & (x['Storage'].isin([module]))
        & (x['Filename'].isin([fournisseur]))
    ]

    def make_clickable(val):
        return '<a href="{}">{}</a>'.format(val, val)

    display(df_.style.format(make_clickable, subset="Source_data"))


# Theme
unique_theme = (df_excel['Theme'].sort_values().unique())
themeW = widgets.Dropdown(options=unique_theme)
init = themeW.value

### Theme: Module

moduleW = widgets.Dropdown(options=(df_excel
                                    .loc[lambda x: x['Theme']
                                         .isin([init])]['Storage']
                                    .sort_values()
                                    .unique()
                                    ))
init2 = moduleW.value


def select_module(theme):
    moduleW.options = (df_excel
                       .loc[lambda x: x['Theme']
                            .isin([theme])]['Storage']
                       .sort_values()
                       .unique()
                       )


# Select fournisseur
fournisseurtW = widgets.Dropdown(options=(df_excel
                                          .loc[lambda x: 
                                               (x['Theme']
                                               .isin([init]))
                                              & (x['Storage']
                                               .isin([init2]))
                                              ]['Filename']
                                          .sort_values()
                                          .unique()
                                          ))


def select_fournisseur(theme, module):
    fournisseurtW.options = (df_excel
                             .loc[lambda x: 
                                               (x['Theme']
                                               .isin([theme]))
                                              & (x['Storage']
                                               .isin([module]))
                                              ]['Filename']
                             .sort_values()
                             .unique()
                             )


i = widgets.interactive(select_module, theme=themeW)

j = widgets.interactive(
    return_df,
    theme=themeW,
    module=moduleW,
    fournisseur=fournisseurtW)

k = widgets.interactive(select_fournisseur, theme = themeW, module=moduleW)

# App Micro

In [ ]:
display(j)